In [ ]:
import pandas as pd
import numpy as np

# What is Time Series Data?

Important to consider progression of time
> - Is the temporal information a key focus of the data?  

## Examples

- Stock prices
- Temperature over the year
- Atmoshperic changes over the course of decades

## Loading in time series

In [ ]:
# Load and display
df = pd.read_csv("min_temp.csv")
display(df.head(10))
display(df.info())

## Make data readable as a datetime

In [ ]:
# Creating a proper datetime using the string formatting
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
# Make the temporal data as the focus
df = df.set_index('Date')

In [ ]:
display(df.head(10))
display(df.info())

# Resampling

## Downsampling

## Upsampling

# Visualizing Time Series

## Showing Changes Over Time

### Line Plot

### Dot Plot

### Grouping Plots

## Showing Distributions

### Histogram

### Density

### Box Plot

### Heat Maps

# Types of Trends

# Removing Trends

# Time Series Decomposition